<a href="https://colab.research.google.com/github/SantiAp11/saber11-analisis-exploratorio/blob/main/analisis_icfes_2014_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Resultados del ICFES Saber 11 en Colombia (2014–2024)

Este proyecto realiza un análisis exploratorio completo de los resultados del examen **Saber 11** en Colombia desde el año **2014** (cuando se implementó el nuevo modelo de evaluación) hasta el año **2024**.

A través del uso de técnicas de análisis de datos, se busca comprender las tendencias, distribuciones y variaciones de los puntajes obtenidos por los estudiantes a lo largo del país.

---

## Objetivos del Proyecto

1. **Explorar y analizar los puntajes globales** del examen Saber 11 por año.
2. **Comparar el rendimiento académico** por **departamento** y **municipio**.
3. Analizar los puntajes por **áreas evaluadas**: Lectura Crítica, Matemáticas, Sociales y Ciudadanas, Ciencias Naturales e Inglés.
4. Identificar los **diez puntajes más altos** obtenidos en cada año (2014–2024).
5. Calcular el **promedio general por año** para detectar posibles tendencias o anomalías.
6. Construir visualizaciones claras para facilitar la interpretación de los resultados.

---

## Fuente de Datos

Los datos utilizados provienen de los archivos oficiales de resultados publicados por el **ICFES** y han sido procesados a partir de archivos de texto plano (.txt) para cada año y convocatoria.

---

## Preguntas Clave de Análisis

- ¿Cuál ha sido la evolución del puntaje global promedio entre 2014 y 2024?
- ¿Qué departamentos y municipios han mostrado un mejor desempeño en el examen?
- ¿Qué estudiantes obtuvieron los puntajes más altos cada año?
- ¿Cómo se han comportado los resultados por área de conocimiento?
- ¿Existe alguna tendencia que sugiera mejoras o retrocesos en el rendimiento académico?

---

## Herramientas Utilizadas

- Python
  - pandas
  - matplotlib / seaborn
- Google Colab
- Google Drive (almacenamiento de archivos)
- Tableau (Dashboard)

---

> Este análisis se desarrolla como parte del portafolio de proyectos de ciencia de datos del autor, con fines exploratorios, educativos y de interés público.


### Paso 1: Montar Google Drive

Este bloque de código monta tu Google Drive para que el notebook pueda acceder a los archivos almacenados allí. Es necesario para poder leer los archivos de resultados del ICFES desde la ruta especificada.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Paso 2: Analizar columnas del archivo fuente

Para determinar cuáles columnas son relevantes para nuestro análisis, leeremos un archivo de ejemplo (`Examen_Saber_11_20141.txt`) y mostraremos la lista completa de sus columnas. Esto nos ayudará a identificar los nombres exactos y decidir cuáles conservar.

In [2]:
import pandas as pd
import os

# Ruta del archivo de ejemplo (usamos 20141 para ver la estructura inicial)
archivo_ejemplo = '/content/drive/MyDrive/Proyectos Análisis de datos/ICFES/ICFES/Examen_Saber_11_20141.txt'

try:
    # Leer solo las primeras filas para obtener los nombres de las columnas
    # Usamos sep=';' porque ya identificamos que es el separador correcto
    df_ejemplo = pd.read_csv(archivo_ejemplo, sep=';', encoding='latin1', low_memory=False, nrows=0)

    # Limpiar nombres de columnas (quitar espacios y convertir a minúsculas)
    df_ejemplo.columns = df_ejemplo.columns.str.lower().str.strip()

    # Mostrar los nombres de las columnas
    print(f"Columnas disponibles en el archivo de ejemplo ({os.path.basename(archivo_ejemplo)}):")
    print(df_ejemplo.columns.tolist())

except FileNotFoundError:
    print(f"[X] Error: El archivo no fue encontrado en la ruta especificada: {archivo_ejemplo}")
except Exception as e:
    print(f"[X] Ocurrió un error al leer el archivo: {e}")

Columnas disponibles en el archivo de ejemplo (Examen_Saber_11_20141.txt):
['periodo', 'estu_consecutivo', 'estu_estudiante', 'cole_area_ubicacion', 'cole_bilingue', 'cole_calendario', 'cole_caracter', 'cole_cod_dane_establecimiento', 'cole_cod_dane_sede', 'cole_cod_depto_ubicacion', 'cole_cod_mcpio_ubicacion', 'cole_codigo_icfes', 'cole_depto_ubicacion', 'cole_genero', 'cole_jornada', 'cole_mcpio_ubicacion', 'cole_naturaleza', 'cole_nombre_establecimiento', 'cole_nombre_sede', 'cole_sede_principal', 'desemp_ingles', 'desemp_profundiza_biologia', 'desemp_profundiza_csociales', 'desemp_profundiza_lenguaje', 'desemp_profundiza_matematica', 'estu_anomatriculaprimero', 'estu_anomatriculasexto', 'estu_anoscolegioactual', 'estu_anospreescolar', 'estu_anoterminobachiller', 'estu_anoterminoquinto', 'estu_antecedentes', 'estu_areareside', 'estu_cod_depto_presentacion', 'estu_cod_iesdeseada', 'estu_cod_mcpio_presentacion', 'estu_cod_mcpioiesdeseada', 'estu_cod_programadeseado', 'estu_cod_reside_

### Paso 3: Cargar y filtrar datos de todos los archivos

Este bloque de código itera sobre todos los archivos `.txt` en la ruta especificada, lee cada archivo (usando el separador `;`), limpia los nombres de las columnas y filtra los archivos anteriores a la convocatoria 2 de 2014 (20142). Para los archivos restantes, selecciona únicamente las columnas base y las columnas de puntaje general (que son consistentes desde 20142 en adelante) y añade columnas para el Año y la Convocatoria. Cada archivo procesado se almacena como un DataFrame en la lista `dataframes_procesados`.

In [3]:
import pandas as pd
import glob
import os
import re

# Ruta a tus archivos
ruta = '/content/drive/MyDrive/Proyectos Análisis de datos/ICFES/ICFES/*.txt'

# Columnas base que vamos a conservar
# Basado en decisiones anteriores, excluyendo: cole_cod_dane_establecimiento, estu_tipo_documento, estu_fecha_nacimiento, estu_etnia, estu_genero
columnas_base = [
    # Estudiante
    'estu_consecutivo',
    # 'estu_genero', # Excluida
    # 'estu_etnia', # Excluida
    # 'estu_fechanacimiento', # Excluida
    # 'estu_tipodocumento', # Excluida

    # Colegio
    # 'cole_cod_dane_establecimiento', # Excluida
    'cole_nombre_establecimiento',
    'cole_caracter',
    'cole_naturaleza',
    'cole_area_ubicacion',
    'cole_calendario',
    'cole_bilingue',
    'cole_mcpio_ubicacion',
    'cole_depto_ubicacion',

    # Ubicación y tiempo
    'periodo',
    'estu_mcpio_presentacion',
    'estu_depto_presentacion',
]

# Columnas de puntaje para los exámenes calificados sobre 500 puntos (desde 20142 en adelante)
# Estas son las áreas que el usuario confirmó: Lectura Crítica, Matemáticas, Sociales y Ciudadanas, Ciencias Naturales e Inglés, más el puntaje global.
columnas_puntaje_general = [
    'punt_global',
    'punt_lectura_critica',
    'punt_matematicas',
    'punt_ingles',
    'punt_c_naturales',
    'punt_sociales_ciudadanas'
]

# Lista para guardar los DataFrames procesados
dataframes_procesados = []

# Leer todos los archivos y procesarlos
for archivo in sorted(glob.glob(ruta)):
    nombre = os.path.basename(archivo)
    print(f"Evaluando archivo: {nombre}")

    # Extraer Año y Convocatoria del nombre del archivo
    match = re.search(r'(\d{4})(\d)', nombre)
    if match:
        anio = int(match.group(1))
        convocatoria = int(match.group(2))
    else:
         print(f"[!] No se encontró año o convocatoria en {nombre}, saltando.")
         continue

    # --- FILTRAR ARCHIVOS ANTERIORES A 20142 (Excluir 20141 y años anteriores) ---
    if anio < 2014 or (anio == 2014 and convocatoria == 1):
        print(f"[-] Saltando archivo anterior a 20142: {nombre}")
        continue
    # ----------------------------------------------------------------------------

    print(f"Procesando archivo: {nombre}")

    try:
        # Leer el archivo con el separador correcto (';' encontrado en 2014, asumimos que es el mismo para todos)
        df = pd.read_csv(archivo, sep=';', encoding='latin1', low_memory=False)

        # Limpiar nombres de columnas (quitar espacios y convertir a minúsculas)
        df.columns = df.columns.str.lower().str.strip()

        # Seleccionar el conjunto de columnas a conservar: base + puntajes generales
        columnas_a_conservar_archivo = columnas_base + columnas_puntaje_general

        # Seleccionar solo las columnas que queremos conservar
        # Asegurarse de que las columnas existen en el DataFrame actual
        columnas_existentes = [col for col in columnas_a_conservar_archivo if col in df.columns]
        df_filtrado = df[columnas_existentes].copy() # Use .copy() to avoid SettingWithCopyWarning

        # Añadir Año y Convocatoria como columnas (ya extraídos arriba)
        match = re.search(r'(\d{4})(\d)', nombre)
        if match:
            df_filtrado['Año'] = int(match.group(1))
            df_filtrado['Convocatoria'] = int(match.group(2))
        else:
             # Esto no debería pasar si el match para 'anio' funcionó, pero se mantiene por seguridad
             print(f"[!] No se encontró año o convocatoria en {nombre}")
             continue

        # Añadir a la lista final
        dataframes_procesados.append(df_filtrado)
        print(f"Procesado: {nombre} ({len(df_filtrado)} filas, {len(df_filtrado.columns)} columnas)")

    except Exception as e:
        print(f"Error procesando {nombre}: {e}")

# Mostrar el número total de dataframes procesados
total_registros = sum(len(df) for df in dataframes_procesados)
print(f"\nTotal de archivos procesados: {len(dataframes_procesados)}")
print(f"Total de registros filtrados: {total_registros}")


# Mostrar las primeras filas del primer dataframe procesado para verificación de columnas
if dataframes_procesados:
    print("\nPrimer DataFrame procesado (Head) para verificar columnas:")
    display(dataframes_procesados[0].head())
    print("\nColumnas del primer DataFrame procesado:")
    print(dataframes_procesados[0].columns.tolist()) # Explicitly show column names

Evaluando archivo: Examen_Saber_11_20141.txt
[-] Saltando archivo anterior a 20142: Examen_Saber_11_20141.txt
Evaluando archivo: Examen_Saber_11_20142.txt
Procesando archivo: Examen_Saber_11_20142.txt
Procesado: Examen_Saber_11_20142.txt (574259 filas, 20 columnas)
Evaluando archivo: Examen_Saber_11_20151.txt
Procesando archivo: Examen_Saber_11_20151.txt
Procesado: Examen_Saber_11_20151.txt (108259 filas, 20 columnas)
Evaluando archivo: Examen_Saber_11_20152.txt
Procesando archivo: Examen_Saber_11_20152.txt
Procesado: Examen_Saber_11_20152.txt (574158 filas, 20 columnas)
Evaluando archivo: Examen_Saber_11_20161.txt
Procesando archivo: Examen_Saber_11_20161.txt
Procesado: Examen_Saber_11_20161.txt (74224 filas, 20 columnas)
Evaluando archivo: Examen_Saber_11_20162.txt
Procesando archivo: Examen_Saber_11_20162.txt
Procesado: Examen_Saber_11_20162.txt (589593 filas, 20 columnas)
Evaluando archivo: Examen_Saber_11_20171.txt
Procesando archivo: Examen_Saber_11_20171.txt
Procesado: Examen_Sa

,estu_consecutivo,cole_nombre_establecimiento,cole_caracter,cole_naturaleza,cole_area_ubicacion,cole_calendario,cole_bilingue,cole_mcpio_ubicacion,cole_depto_ubicacion,periodo,estu_mcpio_presentacion,estu_depto_presentacion,punt_global,punt_lectura_critica,punt_matematicas,punt_ingles,punt_c_naturales,punt_sociales_ciudadanas,Año,Convocatoria
0,SB11201420348324,JUANCHACO,TÃCNICO/ACADÃMICO,OFICIAL,RURAL,A,N,BUENAVENTURA,VALLE,20142,BUENAVENTURA,VALLE,274,61,47,41,53,63,2014,2
1,SB11201420159554,IE JOAQUIN URRUTIA,TÃCNICO/ACADÃMICO,OFICIAL,URBANO,A,N,MEDIO SAN JUAN,CHOCO,20142,ISTMINA,CHOCO,222,40,45,48,43,48,2014,2
2,SB11201420457464,I. E. JUAN PABLO II,TÃCNICO/ACADÃMICO,OFICIAL,URBANO,A,N,VILLAVICENCIO,META,20142,VILLAVICENCIO,META,358,72,74,62,71,73,2014,2
3,SB11201420593164,INSTITUTO DE EDUCACION TECNICA ICET CENTRO,TÃCNICO,NO OFICIAL,URBANO,A,N,CALI,VALLE,20142,CALI,VALLE,184,35,36,46,29,44,2014,2
4,SB11201420554408,LIC SAN BERNARDO,ACADÃMICO,NO OFICIAL,URBANO,A,N,BOGOTÃ D.C.,BOGOTA,20142,BOGOTÃ D.C.,BOGOTA,207,36,47,42,38,44,2014,2



Columnas del primer DataFrame procesado:
['estu_consecutivo', 'cole_nombre_establecimiento', 'cole_caracter', 'cole_naturaleza', 'cole_area_ubicacion', 'cole_calendario', 'cole_bilingue', 'cole_mcpio_ubicacion', 'cole_depto_ubicacion', 'periodo', 'estu_mcpio_presentacion', 'estu_depto_presentacion', 'punt_global', 'punt_lectura_critica', 'punt_matematicas', 'punt_ingles', 'punt_c_naturales', 'punt_sociales_ciudadanas', 'Año', 'Convocatoria']


### Paso 4: Concatenar DataFrames

Este bloque de código concatena todos los DataFrames almacenados en la lista `dataframes_procesados` en un único DataFrame llamado `df_combinado`. El `ignore_index=True` reinicia el índice del DataFrame combinado. Después de la concatenación, imprime el head, la lista completa de columnas, información general (`.info()`) y la forma (`.shape`) del DataFrame resultante. Esto es un paso crucial para consolidar los datos de todos los años en una sola estructura para el análisis.

In [4]:
import pandas as pd

# Concatenar todos los dataframes en uno solo
df_combinado = pd.concat(dataframes_procesados, ignore_index=True)

print("--- DataFrame Combinado (Head) ---")
display(df_combinado.head())

print("\n--- Columnas del DataFrame Combinado ---")
print(df_combinado.columns.tolist())

print("\n--- Información del DataFrame Combinado ---")
df_combinado.info()

print("\n--- Forma del DataFrame Combinado (filas, columnas) ---")
print(df_combinado.shape)

--- DataFrame Combinado (Head) ---


,estu_consecutivo,cole_nombre_establecimiento,cole_caracter,cole_naturaleza,cole_area_ubicacion,cole_calendario,cole_bilingue,cole_mcpio_ubicacion,cole_depto_ubicacion,periodo,estu_mcpio_presentacion,estu_depto_presentacion,punt_global,punt_lectura_critica,punt_matematicas,punt_ingles,punt_c_naturales,punt_sociales_ciudadanas,Año,Convocatoria
0,SB11201420348324,JUANCHACO,TÃCNICO/ACADÃMICO,OFICIAL,RURAL,A,N,BUENAVENTURA,VALLE,20142,BUENAVENTURA,VALLE,274,61,47,41.0,53,63,2014,2
1,SB11201420159554,IE JOAQUIN URRUTIA,TÃCNICO/ACADÃMICO,OFICIAL,URBANO,A,N,MEDIO SAN JUAN,CHOCO,20142,ISTMINA,CHOCO,222,40,45,48.0,43,48,2014,2
2,SB11201420457464,I. E. JUAN PABLO II,TÃCNICO/ACADÃMICO,OFICIAL,URBANO,A,N,VILLAVICENCIO,META,20142,VILLAVICENCIO,META,358,72,74,62.0,71,73,2014,2
3,SB11201420593164,INSTITUTO DE EDUCACION TECNICA ICET CENTRO,TÃCNICO,NO OFICIAL,URBANO,A,N,CALI,VALLE,20142,CALI,VALLE,184,35,36,46.0,29,44,2014,2
4,SB11201420554408,LIC SAN BERNARDO,ACADÃMICO,NO OFICIAL,URBANO,A,N,BOGOTÃ D.C.,BOGOTA,20142,BOGOTÃ D.C.,BOGOTA,207,36,47,42.0,38,44,2014,2



--- Columnas del DataFrame Combinado ---
['estu_consecutivo', 'cole_nombre_establecimiento', 'cole_caracter', 'cole_naturaleza', 'cole_area_ubicacion', 'cole_calendario', 'cole_bilingue', 'cole_mcpio_ubicacion', 'cole_depto_ubicacion', 'periodo', 'estu_mcpio_presentacion', 'estu_depto_presentacion', 'punt_global', 'punt_lectura_critica', 'punt_matematicas', 'punt_ingles', 'punt_c_naturales', 'punt_sociales_ciudadanas', 'Año', 'Convocatoria']

--- Información del DataFrame Combinado ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7239505 entries, 0 to 7239504
Data columns (total 20 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   estu_consecutivo             object 
 1   cole_nombre_establecimiento  object 
 2   cole_caracter                object 
 3   cole_naturaleza              object 
 4   cole_area_ubicacion          object 
 5   cole_calendario              object 
 6   cole_bilingue                object 
 7   cole_mcpio_u

### Paso 5: Limpieza de Columnas de Texto

Este bloque de código limpia las columnas de texto que contienen nombres de colegios, municipios y departamentos. Se aplican varias operaciones para estandarizar los nombres, como eliminar espacios extra, convertir a mayúsculas y manejar caracteres especiales (como las tildes que se muestran incorrectamente). Esto es crucial para agrupar y analizar correctamente los datos por ubicación y colegio.

In [5]:
import unicodedata

def limpiar_texto(texto):
    if isinstance(texto, str):
        # Convertir a mayúsculas
        texto = texto.upper()
        # Eliminar espacios al inicio y final
        texto = texto.strip()
        # Reemplazar caracteres comunes con problemas de tildes/codificación
        texto = texto.replace('Ã', 'Á')
        texto = texto.replace('Ã', 'É')
        texto = texto.replace('Ã', 'Í')
        texto = texto.replace('Ã', 'Ó')
        texto = texto.replace('Ã', 'Ú')
        texto = texto.replace('Ã', 'Ñ')
        texto = texto.replace('Ã', 'A') # Para casos como Ã‘ -> Ñ si la anterior falla
        texto = texto.replace('?', '') # Eliminar caracteres extraños si aparecen como '?'
        texto = texto.replace('\xef\xbf\xbd', '') # Eliminar U+FFFD replacement character


        # Normalizar (puede ayudar con otros caracteres)
        # texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')

        return texto
    return texto # Retorna el valor original si no es un string (ej. NaN)


# Aplicar la función de limpieza a las columnas relevantes
columnas_texto = [
    'cole_nombre_establecimiento',
    'cole_mcpio_ubicacion',
    'cole_depto_ubicacion',
    'estu_mcpio_presentacion',
    'estu_depto_presentacion',
    'cole_caracter'
]

for col in columnas_texto:
    if col in df_combinado.columns:
        df_combinado[col] = df_combinado[col].apply(limpiar_texto)
        print(f"Limpieza aplicada a la columna: {col}")
    else:
        print(f"Columna no encontrada para limpieza: {col}")

print("\nPrimeras filas del DataFrame combinado después de la limpieza:")
display(df_combinado.head())

Limpieza aplicada a la columna: cole_nombre_establecimiento
Limpieza aplicada a la columna: cole_mcpio_ubicacion
Limpieza aplicada a la columna: cole_depto_ubicacion
Limpieza aplicada a la columna: estu_mcpio_presentacion
Limpieza aplicada a la columna: estu_depto_presentacion
Limpieza aplicada a la columna: cole_caracter

Primeras filas del DataFrame combinado después de la limpieza:


,estu_consecutivo,cole_nombre_establecimiento,cole_caracter,cole_naturaleza,cole_area_ubicacion,cole_calendario,cole_bilingue,cole_mcpio_ubicacion,cole_depto_ubicacion,periodo,estu_mcpio_presentacion,estu_depto_presentacion,punt_global,punt_lectura_critica,punt_matematicas,punt_ingles,punt_c_naturales,punt_sociales_ciudadanas,Año,Convocatoria
0,SB11201420348324,JUANCHACO,TÉCNICO/ACADÉMICO,OFICIAL,RURAL,A,N,BUENAVENTURA,VALLE,20142,BUENAVENTURA,VALLE,274,61,47,41.0,53,63,2014,2
1,SB11201420159554,IE JOAQUIN URRUTIA,TÉCNICO/ACADÉMICO,OFICIAL,URBANO,A,N,MEDIO SAN JUAN,CHOCO,20142,ISTMINA,CHOCO,222,40,45,48.0,43,48,2014,2
2,SB11201420457464,I. E. JUAN PABLO II,TÉCNICO/ACADÉMICO,OFICIAL,URBANO,A,N,VILLAVICENCIO,META,20142,VILLAVICENCIO,META,358,72,74,62.0,71,73,2014,2
3,SB11201420593164,INSTITUTO DE EDUCACION TECNICA ICET CENTRO,TÉCNICO,NO OFICIAL,URBANO,A,N,CALI,VALLE,20142,CALI,VALLE,184,35,36,46.0,29,44,2014,2
4,SB11201420554408,LIC SAN BERNARDO,ACADÉMICO,NO OFICIAL,URBANO,A,N,BOGOTÁ D.C.,BOGOTA,20142,BOGOTÁ D.C.,BOGOTA,207,36,47,42.0,38,44,2014,2


### Paso 6: Estandarizar columna `cole_bilingue`

Este bloque de código estandariza los valores en la columna `cole_bilingue` del DataFrame combinado (`df_combinado`), reemplazando 'N' por 'NO' y 'S' por 'SI' para hacer los valores más explícitos.

In [6]:
# Mapear los valores de la columna cole_bilingue
mapeo_bilingue = {
    'N': 'NO',
    'S': 'SI',
    'n': 'NO', # Incluir minúsculas por si acaso
    's': 'SI'  # Incluir minúsculas por si acaso
}

# Aplicar el mapeo a la columna
# Usamos .fillna() para manejar posibles valores nulos antes del mapeo, aunque el mapeo también maneja no-strings
df_combinado['cole_bilingue'] = df_combinado['cole_bilingue'].map(mapeo_bilingue).fillna(df_combinado['cole_bilingue'])

# Mostrar los valores únicos y su conteo después de la estandarización para verificar
print("Valores únicos y su conteo en 'cole_bilingue' después de la estandarización:")
display(df_combinado['cole_bilingue'].value_counts(dropna=False))

# Mostrar el head para verificar
print("\nPrimeras filas del DataFrame combinado después de estandarizar 'cole_bilingue':")
display(df_combinado.head())

Valores únicos y su conteo en 'cole_bilingue' después de la estandarización:


,count
cole_bilingue,
NO,5151530
NaN,1988146
SI,99829



Primeras filas del DataFrame combinado después de estandarizar 'cole_bilingue':


,estu_consecutivo,cole_nombre_establecimiento,cole_caracter,cole_naturaleza,cole_area_ubicacion,cole_calendario,cole_bilingue,cole_mcpio_ubicacion,cole_depto_ubicacion,periodo,estu_mcpio_presentacion,estu_depto_presentacion,punt_global,punt_lectura_critica,punt_matematicas,punt_ingles,punt_c_naturales,punt_sociales_ciudadanas,Año,Convocatoria
0,SB11201420348324,JUANCHACO,TÉCNICO/ACADÉMICO,OFICIAL,RURAL,A,NO,BUENAVENTURA,VALLE,20142,BUENAVENTURA,VALLE,274,61,47,41.0,53,63,2014,2
1,SB11201420159554,IE JOAQUIN URRUTIA,TÉCNICO/ACADÉMICO,OFICIAL,URBANO,A,NO,MEDIO SAN JUAN,CHOCO,20142,ISTMINA,CHOCO,222,40,45,48.0,43,48,2014,2
2,SB11201420457464,I. E. JUAN PABLO II,TÉCNICO/ACADÉMICO,OFICIAL,URBANO,A,NO,VILLAVICENCIO,META,20142,VILLAVICENCIO,META,358,72,74,62.0,71,73,2014,2
3,SB11201420593164,INSTITUTO DE EDUCACION TECNICA ICET CENTRO,TÉCNICO,NO OFICIAL,URBANO,A,NO,CALI,VALLE,20142,CALI,VALLE,184,35,36,46.0,29,44,2014,2
4,SB11201420554408,LIC SAN BERNARDO,ACADÉMICO,NO OFICIAL,URBANO,A,NO,BOGOTÁ D.C.,BOGOTA,20142,BOGOTÁ D.C.,BOGOTA,207,36,47,42.0,38,44,2014,2


### Paso 7: Guardar el DataFrame Combinado a CSV

Este bloque de código guarda el DataFrame `df_combinado` (que contiene todos los datos procesados y combinados) en un archivo CSV en Google Drive. Este archivo CSV podrá ser utilizado posteriormente para importar los datos en herramientas de visualización como Power BI o Tableau.

In [7]:
import os

# Define la ruta donde guardarás el archivo CSV en Google Drive
ruta_guardado = '/content/drive/MyDrive/Proyectos Análisis de datos/ICFES/Resultados_ICFES_Saber11_20142024.csv'

try:
    # Asegúrate de que el directorio exista
    directorio_guardado = os.path.dirname(ruta_guardado)
    if not os.path.exists(directorio_guardado):
        os.makedirs(directorio_guardado)
        print(f"Directorio creado: {directorio_guardado}")

    # Guarda el DataFrame en formato CSV
    df_combinado.to_csv(ruta_guardado, index=False, encoding='utf-8')

    print(f"\nDataFrame guardado exitosamente en: {ruta_guardado}")

except Exception as e:
    print(f"[X] Ocurrió un error al guardar el archivo: {e}")


DataFrame guardado exitosamente en: /content/drive/MyDrive/Proyectos Análisis de datos/ICFES/Resultados_ICFES_Saber11_20142024.csv


In [8]:
# Verificar si 'VALLE DEL CAUCA' existe en las columnas de departamento
departamento_buscado = 'VALLE DEL CAUCA'

existe_cole_depto = departamento_buscado in df_combinado['cole_depto_ubicacion'].unique()
existe_estu_depto = departamento_buscado in df_combinado['estu_depto_presentacion'].unique()

if existe_cole_depto:
    print(f"'{departamento_buscado}' fue encontrado en la columna 'cole_depto_ubicacion'.")
else:
    print(f"'{departamento_buscado}' NO fue encontrado en la columna 'cole_depto_ubicacion'.")

if existe_estu_depto:
    print(f"'{departamento_buscado}' fue encontrado en la columna 'estu_depto_presentacion'.")
else:
    print(f"'{departamento_buscado}' NO fue encontrado en la columna 'estu_depto_presentacion'.")

'VALLE DEL CAUCA' NO fue encontrado en la columna 'cole_depto_ubicacion'.
'VALLE DEL CAUCA' NO fue encontrado en la columna 'estu_depto_presentacion'.


# Task
Estandarizar los nombres de departamentos y municipios en las columnas 'cole_depto_ubicacion', 'estu_depto_presentacion', 'cole_mcpio_ubicacion' y 'estu_mcpio_presentacion' del dataframe `df_combinado`, unificando los registros de nombres duplicados para cada departamento y municipio, y guardar el resultado en un archivo CSV.

# **Estandarización de departamentos**:




### Contar el número de departamentos diferentes

Este bloque de código cuenta y muestra el número total de valores únicos (nombres de departamentos diferentes) que existen en las columnas `cole_depto_ubicacion` y `estu_depto_presentacion`.

In [9]:
# Contar el número de valores únicos en 'cole_depto_ubicacion'
num_deptos_cole = df_combinado['cole_depto_ubicacion'].nunique(dropna=False)
print(f"Número de departamentos diferentes en 'cole_depto_ubicacion': {num_deptos_cole}")

# Contar el número de valores únicos en 'estu_depto_presentacion'
num_deptos_estu = df_combinado['estu_depto_presentacion'].nunique(dropna=False)
print(f"Número de departamentos diferentes en 'estu_depto_presentacion': {num_deptos_estu}")

Número de departamentos diferentes en 'cole_depto_ubicacion': 35
Número de departamentos diferentes en 'estu_depto_presentacion': 35


### Listar nombres de departamentos únicos

Este bloque de código muestra una lista de todos los nombres de departamentos únicos encontrados en las columnas `cole_depto_ubicacion` y `estu_depto_presentacion`. Esto ayuda a visualizar las variaciones existentes y planificar la estandarización.

In [10]:
# Obtener la lista de nombres de departamentos únicos en 'cole_depto_ubicacion'
deptos_cole_unicos = df_combinado['cole_depto_ubicacion'].unique().tolist()
print("Nombres de departamentos únicos en 'cole_depto_ubicacion':")
# Convertir a string antes de ordenar para manejar NaNs u otros tipos
print(sorted([str(item) for item in deptos_cole_unicos]))

# Obtener la lista de nombres de departamentos únicos en 'estu_depto_presentacion'
deptos_estu_unicos = df_combinado['estu_depto_presentacion'].unique().tolist()
print("\nNombres de departamentos únicos en 'estu_depto_presentacion':")
# Convertir a string antes de ordenar para manejar NaNs u otros tipos
print(sorted([str(item) for item in deptos_estu_unicos]))

Nombres de departamentos únicos en 'cole_depto_ubicacion':
['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLANTICO', 'BOGOTA', 'BOGOTÁ', 'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA', 'CUNDINAMARCA', 'GUAINIA', 'GUAVIARE', 'HUILA', 'LA GUAJIRA', 'MAGDALENA', 'META', 'NARIÑO', 'NORTE SANTANDER', 'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SAN ANDRES', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE', 'VAUPES', 'VICHADA', 'nan']

Nombres de departamentos únicos en 'estu_depto_presentacion':
['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLANTICO', 'BOGOTA', 'BOGOTÁ', 'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA', 'CUNDINAMARCA', 'GUAINIA', 'GUAVIARE', 'HUILA', 'LA GUAJIRA', 'MAGDALENA', 'META', 'NARIÑO', 'NORTE SANTANDER', 'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SAN ANDRES', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE', 'VAUPES', 'VICHADA', 'nan']


### Paso 8: Estandarizar nombres de Departamentos

Este bloque de código estandariza los nombres de los departamentos en las columnas `cole_depto_ubicacion` y `estu_depto_presentacion` utilizando un diccionario de mapeo para corregir inconsistencias como tildes o abreviaciones.

In [11]:
# Mapeo para estandarizar nombres de departamentos
mapeo_deptos = {
    'BOGOTA': 'BOGOTÁ D.C.', # Unificar 'BOGOTA' bajo 'BOGOTÁ D.C.'
    'VALLE': 'VALLE DEL CAUCA', # Unificar 'VALLE' bajo 'VALLE DEL CAUCA'
    'NARIÃ‘O': 'NARIÑO', # Corregir tilde en NARIÑO
    'CORDOBA': 'CÓRDOBA', # Corregir tilde en CÓRDOBA
    'BOGOTÁ': 'BOGOTÁ D.C.' # Unificar 'BOGOTÁ' bajo 'BOGOTÁ D.C.'
    # Agrega más mapeos si identificas otras inconsistencias
}

# Aplicar el mapeo a las columnas de departamento
columnas_deptos = ['cole_depto_ubicacion', 'estu_depto_presentacion']

for col in columnas_deptos:
    if col in df_combinado.columns:
        df_combinado[col] = df_combinado[col].replace(mapeo_deptos)
        print(f"Estandarización aplicada a la columna: {col}")
    else:
        print(f"Columna no encontrada para estandarizar: {col}")


# Verificar los valores únicos después de la estandarización
print("\nConteo de valores únicos en 'cole_depto_ubicacion' después de estandarizar:")
display(df_combinado['cole_depto_ubicacion'].value_counts(dropna=False))

print("\nConteo de valores únicos en 'estu_depto_presentacion' después de estandarizar:")
display(df_combinado['estu_depto_presentacion'].value_counts(dropna=False))

# Contar el número de valores únicos nuevamente
num_deptos_cole_estandarizado = df_combinado['cole_depto_ubicacion'].nunique(dropna=False)
print(f"\nNúmero de departamentos diferentes en 'cole_depto_ubicacion' después de estandarizar: {num_deptos_cole_estandarizado}")

num_deptos_estu_estandarizado = df_combinado['estu_depto_presentacion'].nunique(dropna=False)
print(f"Número de departamentos diferentes en 'estu_depto_presentacion' después de estandarizar: {num_deptos_estu_estandarizado}")

Estandarización aplicada a la columna: cole_depto_ubicacion
Estandarización aplicada a la columna: estu_depto_presentacion

Conteo de valores únicos en 'cole_depto_ubicacion' después de estandarizar:


,count
cole_depto_ubicacion,
NaN,1008205
BOGOTÁ D.C.,981409
ANTIOQUIA,823615
VALLE DEL CAUCA,506317
CUNDINAMARCA,419360
ATLANTICO,351674
SANTANDER,293661
BOLIVAR,290177
CÓRDOBA,226980



Conteo de valores únicos en 'estu_depto_presentacion' después de estandarizar:


,count
estu_depto_presentacion,
BOGOTÁ D.C.,1151600
ANTIOQUIA,891698
VALLE DEL CAUCA,622351
CUNDINAMARCA,434492
ATLANTICO,392231
SANTANDER,356645
BOLIVAR,304781
NARIÑO,285851
BOYACA,243115



Número de departamentos diferentes en 'cole_depto_ubicacion' después de estandarizar: 34
Número de departamentos diferentes en 'estu_depto_presentacion' después de estandarizar: 34


### Contar el número de departamentos diferentes (después de estandarización)

Este bloque de código cuenta y muestra el número total de valores únicos (nombres de departamentos diferentes) que existen en las columnas `cole_depto_ubicacion` y `estu_depto_presentacion` después de la estandarización.

In [12]:
# Contar el número de valores únicos en 'cole_depto_ubicacion' después de estandarizar
num_deptos_cole_estandarizado = df_combinado['cole_depto_ubicacion'].nunique(dropna=False)
print(f"Número de departamentos diferentes en 'cole_depto_ubicacion' después de estandarizar: {num_deptos_cole_estandarizado}")

# Contar el número de valores únicos en 'estu_depto_presentacion' después de estandarizar
num_deptos_estu_estandarizado = df_combinado['estu_depto_presentacion'].nunique(dropna=False)
print(f"Número de departamentos diferentes en 'estu_depto_presentacion' después de estandarizar: {num_deptos_estu_estandarizado}")

Número de departamentos diferentes en 'cole_depto_ubicacion' después de estandarizar: 34
Número de departamentos diferentes en 'estu_depto_presentacion' después de estandarizar: 34


### Listar nombres de departamentos únicos (después de la primera estandarización)

Este bloque de código muestra una lista de todos los nombres de departamentos únicos encontrados en las columnas `cole_depto_ubicacion` y `estu_depto_presentacion` después de la primera ronda de estandarización. Esto ayuda a identificar inconsistencias restantes.

In [13]:
# Obtener la lista de nombres de departamentos únicos en 'cole_depto_ubicacion'
deptos_cole_unicos = df_combinado['cole_depto_ubicacion'].unique().tolist()
print("Nombres de departamentos únicos en 'cole_depto_ubicacion':")
# Convertir a string antes de ordenar para manejar NaNs u otros tipos
print(sorted([str(item) for item in deptos_cole_unicos]))

# Obtener la lista de nombres de departamentos únicos en 'estu_depto_presentacion'
deptos_estu_unicos = df_combinado['estu_depto_presentacion'].unique().tolist()
print("\nNombres de departamentos únicos en 'estu_depto_presentacion':")
# Convertir a string antes de ordenar para manejar NaNs u otros tipos
print(sorted([str(item) for item in deptos_estu_unicos]))

Nombres de departamentos únicos en 'cole_depto_ubicacion':
['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLANTICO', 'BOGOTÁ D.C.', 'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CUNDINAMARCA', 'CÓRDOBA', 'GUAINIA', 'GUAVIARE', 'HUILA', 'LA GUAJIRA', 'MAGDALENA', 'META', 'NARIÑO', 'NORTE SANTANDER', 'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SAN ANDRES', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE DEL CAUCA', 'VAUPES', 'VICHADA', 'nan']

Nombres de departamentos únicos en 'estu_depto_presentacion':
['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLANTICO', 'BOGOTÁ D.C.', 'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CUNDINAMARCA', 'CÓRDOBA', 'GUAINIA', 'GUAVIARE', 'HUILA', 'LA GUAJIRA', 'MAGDALENA', 'META', 'NARIÑO', 'NORTE SANTANDER', 'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SAN ANDRES', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE DEL CAUCA', 'VAUPES', 'VICHADA', 'nan']


#Diccionario Municipios por departamento


In [14]:
municipios_dict = (
    {
    "AMAZONAS": {
        "MIRITÍ - PARANÁ": ["MIRITÍ - PARANÁ"],
        "PTO SANTANDER": ["PTO SANTANDER", "PUERTO SANTANDER"]
    },
    "ANTIOQUIA": {
        "ALEJANDRÍA": ["ALEJANDRIA", "ALEJANDRÍA"],
        "APARTADÓ": ["APARTADÓ"],
        "BOLÍVAR": ["BOLÍVAR"],
        "CÁCERES": ["CÁCERES"],
        "CAROLINA DEL PRÍNCIPE": ["CAROLINA DEL PRÍNCIPE"],
        "CAÑAS GORDAS": ["CAÑAS GORDAS"],
        "CIUDAD BOLÍVAR": ["CIUDAD BOLÍVAR"],
        "DONMATÍAS": ["DONMATÍAS"],
        "EBÉJICO": ["EBÉJICO"],
        "EL CARMEN DE VIBORAL": ["EL CARMEN DE VIBORAL"],
        "EL PEÑOL": ["EL PEÑOL"],
        "EL RETIRO": ["EL RETIRO"],
        "EL SANTUARIO": ["EL SANTUARIO"],
        "GÓMEZ PLATA": ["GÓMEZ PLATA"],
        "ITAGÜÍ": ["ITAGÜÍ"],
        "MEDELLÍN": ["MEDELLÍN"],
        "MURINDÓ": ["MURINDÓ"],
        "MUTATÁ": ["MUTATÁ"],
        "NECHÍ": ["NECHÍ"],
        "NECOCLÍ": ["NECOCLÍ"],
        "PUEBLORRICO": ["PUEBLORRICO"],
        "PUERTO BERRÍO": ["PUERTO BERRÍO"],
        "PUERTO NARE": ["PUERTO NARE"],
        "PUERTO TRIUNFO": ["PUERTO TRIUNFO"],
        "RIONEGRO": ["RIONEGRO"],
        "SABANETA": ["SABANETA"],
        "SAN ANDRÉS DE CUERQUÍA": ["SAN ANDRÉS DE CUERQUÍA"],
        "SAN JERÓNIMO": ["SAN JERÓNIMO"],
        "SAN JOSÉ DE LA MONTAÑA": ["SAN JOSÉ DE LA MONTAÑA"],
        "SAN JUAN DE URABÁ": ["SAN JUAN DE URABÁ"],
        "SAN LUIS": ["SAN LUIS"],
        "SAN PEDRO DE LOS MILAGROS": ["SAN PEDRO DE LOS MILAGROS"],
        "SAN PEDRO DE URABÁ": ["SAN PEDRO DE URABÁ"],
        "SAN RAFAEL": ["SAN RAFAEL"],
        "SAN ROQUE": ["SAN ROQUE"],
        "SAN VICENTE FERRER": ["SAN VICENTE FERRER"],
        "SANTA BÁRBARA": ["SANTA BÁRBARA"],
        "SANTA ROSA DE OSOS": ["SANTA ROSA DE OSOS"],
        "SANTAFÉ DE ANTIOQUIA": ["SANTAFÉ DE ANTIOQUIA"],
        "SANTO DOMINGO": ["SANTO DOMINGO"],
        "SEGOVIA": ["SEGOVIA"],
        "SONSÓN": ["SONSÓN"],
        "SOPETRÁN": ["SOPETRÁN"],
        "TARAZÁ": ["TARAZÁ"],
        "TÁMESIS": ["TÁMESIS"],
        "URAMITA": ["URAMITA"],
        "URRAO": ["URRAO"],
        "VALDIVIA": ["VALDIVIA"],
        "VALPARAÍSO": ["VALPARAÍSO"],
        "VEGACHÍ": ["VEGACHÍ"],
        "VENECIA": ["VENECIA"],
        "VIGÍA DEL FUERTE": ["VIGÍA DEL FUERTE"],
        "YALÍ": ["YALÍ"],
        "YARUMAL": ["YARUMAL"],
        "YOLOMBÓ": ["YOLOMBÓ"],
        "YONDÓ": ["YONDÓ"],
        "ZARAGOZA": ["ZARAGOZA"],
        "nan": ["nan"]
    },
    "ARAUCA": {
        "PUERTO RONDÓN": ["PUERTO RONDON", "PUERTO RONDÓN"]
    },
    "ATLANTICO": {
        "MANATÍ": ["MANATI", "MANATÍ"],
        "PIOJÓ": ["PIOJO", "PIOJÓ"],
        "REPELÓN": ["REPELON", "REPELÓN"],
        "SANTA LUCÍA": ["SANTA LUCIA", "SANTA LUCÍA"],
        "SANTO TOMÁS": ["SANTO TOMAS", "SANTO TOMÁS"],
        "TUBARÁ": ["TUBARA", "TUBARÁ"],
        "USIACURÍ": ["USIACURI", "USIACURÍ"]
    },
    "BOGOTÁ D.C.": {
        "BOGOTÁ D.C.": ["BOGOTÁ D.C."]
    },
    "BOLIVAR": {
        "ACHÍ": ["ACHI", "ACHÍ"],
        "CÓRDOBA": ["CORDOBA", "CÓRDOBA"],
        "EL CARMEN DE BOLÍVAR": ["EL CARMEN DE BOLIVAR", "EL CARMEN DE BOLÍVAR"],
        "EL PEÑÓN": ["EL PEÑON", "EL PEÑÓN"],
        "MAGANGUÉ": ["MAGANGUE", "MAGANGUÉ"],
        "MARÍA LA BAJA": ["MARIA LA BAJA", "MARÍA LA BAJA"],
        "MOMPÓS": ["MOMPOS", "MOMPÓS"],
        "NOROSÍ": ["NOROSI", "NOROSÍ"],
        "RÍO VIEJO": ["RIO VIEJO", "RÍO VIEJO"],
        "SAN CRISTÓBAL": ["SAN CRISTOBAL", "SAN CRISTÓBAL"],
        "SAN MARTÍN DE LOBA": ["SAN MARTIN DE LOBA", "SAN MARTÍN DE LOBA"],
        "SIMITÍ": ["SIMITI", "SIMITÍ"],
        "TURBANÁ": ["TURBANA", "TURBANÁ"],
        "TIQUISIO (PUERTO RICO)": ["TIQUISIO", "TIQUISIO (PUERTO RICO)"] # Se incluye la variante más específica
    },
    "BOYACA": {
        "BELÉN": ["BELEN", "BELÉN"],
        "BETÉITIVA": ["BETEITIVA", "BETÉITIVA"],
        "BOYACÁ": ["BOYACA", "BOYACÁ"],
        "BUSBANZÁ": ["BUSBANZA", "BUSBANZÁ"],
        "CHIQUINQUIRÁ": ["CHIQUINQUIRA", "CHIQUINQUIRÁ"],
        "CHÍQUIZA": ["CHIQUIZA", "CHÍQUIZA"],
        "CHIVATÁ": ["CHIVATA", "CHIVATÁ"],
        "CIÉNEGA": ["CIENEGA", "CIÉNEGA"],
        "CÓMBITA": ["COMBITA", "CÓMBITA"],
        "COVARACHÍA": ["COVARACHIA", "COVARACHÍA"],
        "CUBARÁ": ["CUBARA", "CUBARÁ"],
        "CUÍTIVA": ["CUITIVA", "CUÍTIVA"],
        "GACHANTIVÁ": ["GACHANTIVA", "GACHANTIVÁ"],
        "GÁMEZA": ["GAMEZA", "GÁMEZA"],
        "GA\x9cICÁN DE LA SIERRA": ["GA\x9cICÁN DE LA SIERRA"], # Conserva el carácter especial
        "GUAYATÁ": ["GUAYATA", "GUAYATÁ"],
        "GUICÁN": ["GUICAN", "GUICÁN"],
        "MARIPÍ": ["MARIPI", "MARIPÍ"],
        "MONGUÍ": ["MONGUI", "MONGUÍ"],
        "MONIQUIRÁ": ["MONIQUIRA", "MONIQUIRÁ"],
        "NUEVO COLÓN": ["NUEVO COLON", "NUEVO COLÓN"],
        "OICATÁ": ["OICATA", "OICATÁ"],
        "PÁEZ": ["PAEZ", "PÁEZ"],
        "PAZ DE RÍO": ["PAZ DE RIO", "PAZ DE RÍO"],
        "PUERTO BOYACÁ": ["PUERTO BOYACA", "PUERTO BOYACÁ"],
        "QUÍPAMA": ["QUIPAMA", "QUÍPAMA"],
        "RAMIRIQUÍ": ["RAMIRIQUI", "RAMIRIQUÍ"],
        "RÁQUIRA": ["RAQUIRA", "RÁQUIRA"],
        "RONDÓN": ["RONDON", "RONDÓN"],
        "SABOYÁ": ["SABOYA", "SABOYÁ"],
        "SÁCHICA": ["SACHICA", "SÁCHICA"],
        "SAMACÁ": ["SAMACA", "SAMACÁ"],
        "SAN JOSÉ DE PARE": ["SAN JOSE DE PARE", "SAN JOSÉ DE PARE"],
        "SANTA MARÍA": ["SANTA MARIA", "SANTA MARÍA"],
        "SANTA SOFÍA": ["SANTA SOFIA", "SANTA SOFÍA"],
        "SOATÁ": ["SOATA", "SOATÁ"],
        "SOCOTÁ": ["SOCOTA", "SOCOTÁ"],
        "SORACÁ": ["SORACA", "SORACÁ"],
        "SOTAQUIRÁ": ["SOTAQUIRA", "SOTAQUIRÁ"],
        "SUSACÓN": ["SUSACON", "SUSACÓN"],
        "SUTAMARCHÁN": ["SUTAMARCHAN", "SUTAMARCHÁN"],
        "TIBANÁ": ["TIBANA", "TIBANÁ"],
        "TINJACÁ": ["TINJACA", "TINJACÁ"],
        "TOGA\x9cÍ": ["TOGA\x9cÍ", "TOGUI"],
        "TÓPAGA": ["TOPAGA", "TÓPAGA"],
        "TUNUNGUÁ": ["TUNUNGUA", "TUNUNGUÁ"],
        "TURMEQUÉ": ["TURMEQUE", "TURMEQUÉ"],
        "TUTAZÁ": ["TUTASA", "TUTAZÁ"],
        "ÚMBITA": ["UMBITA", "ÚMBITA"],
        "VIRACACHÁ": ["VIRACACHA", "VIRACACHÁ"]
    },
    "CALDAS": {
        "BELALCÁZAR": ["BELALCAZAR", "BELALCÁZAR"],
        "CHINCHINÁ": ["CHINCHINA", "CHINCHINÁ"],
        "PÁCORA": ["PACORA", "PÁCORA"],
        "SAMANÁ": ["SAMANA", "SAMANÁ"],
        "SAN JOSÉ": ["SAN JOSE", "SAN JOSÉ"],
        "SUPÍA": ["SUPIA", "SUPÍA"],
        "VILLAMARÍA": ["VILLAMARIA", "VILLAMARÍA"]
    },
    "CAQUETA": {
        "BELÉN DE LOS ANDAQUÍES": ["BELEN DE LOS ANDAQUIES", "BELÉN DE LOS ANDAQUÍES"],
        "CARTAGENA DEL CHAIRÁ": ["CARTAGENA DEL CHAIRA", "CARTAGENA DEL CHAIRÁ"],
        "EL PAUJÍL": ["EL PAUJIL", "EL PAUJÍL"],
        "MILÁN": ["MILAN", "MILÁN"],
        "SAN JOSÉ DEL FRAGUA": ["SAN JOSE DEL FRAGUA", "SAN JOSÉ DEL FRAGUA"],
        "SAN VICENTE DEL CAGUÁN": ["SAN VICENTE DEL CAGUAN", "SAN VICENTE DEL CAGUÁN"],
        "VALPARAÍSO": ["VALPARAÍSO"]
    },
    "CASANARE": {
        "CHÁMEZA": ["CHAMEZA", "CHÁMEZA"],
        "MANÍ": ["MANI", "MANÍ"],
        "NUNCHÍA": ["NUNCHIA", "NUNCHÍA"],
        "OROCUÉ": ["OROCUE", "OROCUÉ"],
        "SÁCAMA": ["SACAMA", "SÁCAMA"],
        "TÁMARA": ["TAMARA", "TÁMARA"]
    },
    "CAUCA": {
        "BOLÍVAR": ["BOLIVAR", "BOLÍVAR"],
        "CAJIBÍO": ["CAJIBIO", "CAJIBÍO"],
        "GUAPÍ": ["GUAPI", "GUAPÍ"],
        "INZÁ": ["INZA", "INZÁ"],
        "JAMBALÓ": ["JAMBALO", "JAMBALÓ"],
        "LÓPEZ DE MICAY": ["LOPEZ (MICAY)", "LÓPEZ DE MICAY"],
        "PATÍA": ["PATIA(EL BORDO)", "PATÍA"],
        "PIENDAMÓ - TUNÍA": ["PIENDAMO", "PIENDAMÓ - TUNÍA"],
        "POPAYÁN": ["POPAYAN", "POPAYÁN"],
        "PURACÉ": ["PURACE (COCONUCO)", "PURACÉ"],
        "PÁEZ": ["PAEZ (BELALCAZAR)", "PÁEZ"],
        "SAN SEBASTIÁN": ["SAN SEBASTIAN", "SAN SEBASTIÁN"],
        "SOTARA": ["SOTARA", "SOTARA (PAISPAMBA)"],
        "SUÁREZ": ["SUAREZ", "SUÁREZ"],
        "TIMBÍO": ["TIMBIO", "TIMBÍO"],
        "TIMBIQUÍ": ["TIMBIQUI", "TIMBIQUÍ"],
        "TORIBÍO": ["TORIBIO", "TORIBÍO"],
        "TOTORÓ": ["TOTORO", "TOTORÓ"]
    },
    "CESAR": {
        "AGUSTÍN CODAZZI": ["AGUSTIN CODAZZI", "AGUSTÍN CODAZZI"],
        "CHIRIGUANÁ": ["CHIRIGUANA", "CHIRIGUANÁ"],
        "CURUMANÍ": ["CURUMANI", "CURUMANÍ"],
        "GONZÁLEZ": ["GONZALEZ", "GONZÁLEZ"],
        "LA PAZ (ROBLES)": ["LA PAZ", "LA PAZ (ROBLES)"], # Manteniendo la variante más específica
        "MANAURE BALCÓN DEL CESAR": ["MANAURE BALCON DEL CESAR", "MANAURE BALCÓN DEL CESAR"],
        "RÍO DE ORO": ["RIO DE ORO", "RÍO DE ORO"],
        "SAN MARTÍN": ["SAN MARTIN", "SAN MARTÍN"]
    },
    "CHOCO": {
        "ACANDÍ": ["ACANDI", "ACANDÍ"],
        "ALTO BAUDÓ": ["ALTO BAUDO (PIE DE PATO)", "ALTO BAUDÓ"],
        "ATRATO": ["ATRATO", "ATRATO (YUTO)"],
        "BAGADÓ": ["BAGADO", "BAGADÓ"],
        "BAHÍA SOLANO": ["BAHIA SOLANO (MUTIS)", "BAHÍA SOLANO"],
        "BAJO BAUDÓ": ["BAJO BAUDO\x93 (PIZARRO)", "BAJO BAUDÓ"],
        "BELÉN DE BAJIRÁ": ["BELEN DE BAJIRA", "BELÉN DE BAJIRÁ"],
        "BOJAYÁ": ["BOJAYA (BELLAVISTA)", "BOJAYÁ"],
        "CARMEN DEL DARIÉN": ["CARMEN DEL DARIEN", "CARMEN DEL DARIÉN"],
        "CÉRTEGUI": ["CERTEGUI", "CÉRTEGUI"],
        "EL CANTÓN DEL SAN PABLO": ["CANTON DEL SAN PABLO", "EL CANTÓN DEL SAN PABLO"],
        "JURADÓ": ["JURADO", "JURADÓ"],
        "LLORÓ": ["LLORO", "LLORÓ"],
        "MEDIO BAUDÓ": ["MEDIO BAUDO", "MEDIO BAUDÓ"],
        "NÓVITA": ["NOVITA", "NÓVITA"],
        "NUQUÍ": ["NUQUI", "NUQUÍ"],
        "RÍO IRÓ": ["RIO IRO", "RÍO IRÓ"],
        "RÍO QUITO": ["RIO QUITO", "RÍO QUITO"],
        "SAN JOSÉ DEL PALMAR": ["SAN JOSE DEL PALMAR", "SAN JOSÉ DEL PALMAR"],
        "SIPÍ": ["SIPI", "SIPÍ"],
        "TADÓ": ["TADO", "TADÓ"],
        "UNIÓN PANAMERICANA": ["UNION PANAMERICANA", "UNIÓN PANAMERICANA"]
    },
    "CUNDINAMARCA": {
        "ALBÁN": ["ALBAN", "ALBÁN"],
        "ARBELÁEZ": ["ARBELAEZ", "ARBELÁEZ"],
        "BELTRÁN": ["BELTRAN", "BELTRÁN"],
        "CAPARRAPÍ": ["CAPARRAPI", "CAPARRAPÍ"],
        "CÁQUEZA": ["CAQUEZA", "CÁQUEZA"],
        "CHAGUANÍ": ["CHAGUANI", "CHAGUANÍ"],
        "CHÍA": ["CHIA", "CHÍA"],
        "CHOACHÍ": ["CHOACHI", "CHOACHÍ"],
        "CHOCONTÁ": ["CHOCONTA", "CHOCONTÁ"],
        "CUCUNUBÁ": ["CUCUNUBA", "CUCUNUBÁ"],
        "EL PEÑÓN": ["EL PEÑON", "EL PEÑÓN"],
        "FACATATIVÁ": ["FACATATIVA", "FACATATIVÁ"],
        "FÓMEQUE": ["FOMEQUE", "FÓMEQUE"],
        "FÚQUENE": ["FUQUENE", "FÚQUENE"],
        "GACHALÁ": ["GACHALA", "GACHALÁ"],
        "GACHANCIPÁ": ["GACHANCIPA", "GACHANCIPÁ"],
        "GACHETÁ": ["GACHETA", "GACHETÁ"],
        "GUACHETÁ": ["GUACHETA", "GUACHETÁ"],
        "GUATAQUÍ": ["GUATAQUI", "GUATAQUÍ"],
        "GUAYABAL DE SÍQUIMA": ["GUAYABAL DE SIQUIMA", "GUAYABAL DE SÍQUIMA"],
        "GUTIÉRREZ": ["GUTIERREZ", "GUTIÉRREZ"],
        "JERUSALÉN": ["JERUSALEN", "JERUSALÉN"],
        "JUNÍN": ["JUNIN", "JUNÍN"],
        "MACHETÁ": ["MACHETA", "MACHETÁ"],
        "NEMOCÓN": ["NEMOCON", "NEMOCÓN"],
        "PULÍ": ["PULI", "PULÍ"],
        "SAN ANTONIO DE TEQUENDAMA": ["SAN ANTONIO DE TEQUENDAMA", "SAN ANTONIO DEL TEQUENDAMA"],
        "SAN JUAN DE RIOSECO": ["SAN JUAN DE RIO SECO", "SAN JUAN DE RIOSECO"],
        "SESQUILÉ": ["SESQUILE", "SESQUILÉ"],
        "SIBATÉ": ["SIBATE", "SIBATÉ"],
        "SOPÓ": ["SOPO", "SOPÓ"],
        "SUPATÁ": ["SUPATA", "SUPATÁ"],
        "TIBANÁ": ["TIBANA", "TIBANÁ"],
        "TOCANCIPÁ": ["TOCANCIPA", "TOCANCIPÁ"],
        "TOPAIPÍ": ["TOPAIPI", "TOPAIPÍ"],
        "UBALÁ": ["UBALA", "UBALÁ"],
        "ÚTICA": ["UTICA", "ÚTICA"],
        "VENECIA": ["VENECIA", "VENECIA (OSPINA PEREZ)"],
        "VIANÍ": ["VIANI", "VIANÍ"],
        "VILLAGÓMEZ": ["VILLAGOMEZ", "VILLAGÓMEZ"],
        "VILLAPINZÓN": ["VILLAPINZON", "VILLAPINZÓN"],
        "VIOTÁ": ["VIOTA", "VIOTÁ"],
        "YACOPÍ": ["YACOPI", "YACOPÍ"],
        "ZIPACÓN": ["ZIPACON", "ZIPACÓN"],
        "ZIPAQUIRÁ": ["ZIPAQUIRA", "ZIPAQUIRÁ"]
    },
    "CÓRDOBA": {
        "CERETÉ": ["CERETE", "CERETÉ"],
        "CHIMÁ": ["CHIMA", "CHIMÁ"],
        "CHINÚ": ["CHINU", "CHINÚ"],
        "CIÉNAGA DE ORO": ["CIENAGA DE ORO", "CIÉNAGA DE ORO"],
        "LA APARTADA": ["LA APARTADA", "LA APARTADA (LA FRONTERA)"],
        "MONTELÍBANO": ["MONTELIBANO", "MONTELÍBANO"],
        "PURÍSIMA": ["PURISIMA", "PURÍSIMA"],
        "SAHAGÚN": ["SAHAGUN", "SAHAGÚN"],
        "SAN ANDRÉS DE SOTAVENTO": ["SAN ANDRES SOTAVENTO", "SAN ANDRÉS DE SOTAVENTO"],
        "SAN JOSÉ DE URÉ": ["SAN JOSE DE URE", "SAN JOSÉ DE URÉ"]
    },
    "GUAINIA": {
        "INÍRIDA": ["INIRIDA", "INÍRIDA"]
    },
    "GUAVIARE": {
        "SAN JOSÉ DEL GUAVIARE": ["SAN JOSÉ DEL GUAVIARE"]
    },
    "HUILA": {
        "ELÍAS": ["ELIAS", "ELÍAS"],
        "GARZÓN": ["GARZON", "GARZÓN"],
        "NÁTAGA": ["NATAGA", "NÁTAGA"],
        "SAN AGUSTÍN": ["SAN AGUSTIN", "SAN AGUSTÍN"],
        "SANTA MARÍA": ["SANTA MARIA", "SANTA MARÍA"],
        "TIMANÁ": ["TIMANA", "TIMANÁ"],
        "YAGUARÁ": ["YAGUARA", "YAGUARÁ"],
        "ÍQUIRA": ["IQUIRA", "ÍQUIRA"]
    },
    "LA GUAJIRA": {
        "DISTRACCIÓN": ["DISTRACCION", "DISTRACCIÓN"]
    },
    "MAGDALENA": {
        "ARIGUANÍ": ["ARIGUANI (EL DIFICIL)", "ARIGUANÍ"],
        "CERRO SAN ANTONIO": ["CERRO DE SAN ANTONIO", "CERRO SAN ANTONIO"],
        "CIÉNAGA": ["CIENAGA", "CIÉNAGA"],
        "EL PIÑÓN": ["EL PIÑON", "EL PIÑÓN"],
        "EL RETÉN": ["EL RETEN", "EL RETÉN"],
        "FUNDACIÓN": ["FUNDACION", "FUNDACIÓN"],
        "SABANAS DE SAN ÁNGEL": ["SABANAS DE SAN ANGEL", "SABANAS DE SAN ÁNGEL"],
        "SAN SEBASTIÁN DE BUENAVISTA": ["SAN SEBASTIAN DE BUENAVISTA", "SAN SEBASTIÁN DE BUENAVISTA"],
        "SAN ZENÓN": ["SAN ZENON", "SAN ZENÓN"],
        "SANTA BÁRBARA DE PINTO": ["SANTA BARBARA DE PINTO", "SANTA BÁRBARA DE PINTO"],
        "ZAPAYÁN": ["ZAPAYAN", "ZAPAYÁN"]
    },
    "META": {
        "ACACÍAS": ["ACACIAS", "ACACÍAS"],
        "BARRANCA DE UPÍA": ["BARRANCA DE UPIA", "BARRANCA DE UPÍA"],
        "FUENTEDEORO": ["FUENTE DE ORO", "FUENTEDEORO"],
        "LEJANÍAS": ["LEJANIAS", "LEJANÍAS"],
        "MAPIRIPÁN": ["MAPIRIPAN", "MAPIRIPÁN"],
        "PUERTO GAITÁN": ["PUERTO GAITAN", "PUERTO GAITÁN"],
        "PUERTO LÓPEZ": ["PUERTO LOPEZ", "PUERTO LÓPEZ"],
        "SAN MARTÍN": ["SAN MARTIN", "SAN MARTÍN"],
        "VISTAHERMOSA": ["VISTA HERMOSA", "VISTAHERMOSA"]
    },
    "NARIÑO": {
        "ALBÁN": ["ALBAN (SAN JOSE)", "ALBÁN"],
        "ANCUYÁ": ["ANCUYA", "ANCUYÁ"],
        "ARBOLEDA": ["ARBOLEDA", "ARBOLEDA (BERRUECOS)"],
        "BELÉN": ["BELEN", "BELÉN"],
        "CHACHAGUI": ["CHACHAGA\x9cÍ", "CHACHAGUI"],
        "COLÓN": ["COLON (GENOVA)", "COLÓN"],
        "CONSACÁ": ["CONSACA", "CONSACÁ"],
        "CÓRDOBA": ["CORDOBA", "CÓRDOBA"],
        "CUASPÚD": ["CUASPUD (CARLOSAMA)", "CUASPÚD"],
        "CUMBITARA": ["CUMBITARA", "CUMBITARA"], # No hay tilde, pero se agrupa
        "EL TABLÓN DE GÓMEZ": ["EL TABLON", "EL TABLÓN DE GÓMEZ"],
        "GUAITARILLA": ["GUAITARILLA", "GUAITARILLA"], # No hay tilde
        "GUALMATÁN": ["GUALMATAN", "GUALMATÁN"],
        "IMUÉS": ["IMUES", "IMUÉS"],
        "LOS ANDES": ["LOS ANDES", "LOS ANDES (SOTOMAYOR)"],
        "MAGA\x9cÍ": ["MAGA\x9cÍ", "MAGUI (PAYAN)"], # Conserva el carácter especial
        "MALLAMA": ["MALLAMA", "MALLAMA (PIEDRANCHA)"],
        "POTOSÍ": ["POTOSI", "POTOSÍ"],
        "ROBERTO PAYÁN": ["ROBERTO PAYAN (SAN JOSE)", "ROBERTO PAYÁN"],
        "SANDONÁ": ["SANDONA", "SANDONÁ"],
        "SANTA BÁRBARA": ["SANTA BARBARA (ISCUANDE)", "SANTA BÁRBARA"],
        "SANTACRUZ": ["SANTACRUZ", "SANTACRUZ (GUACHAVES)"],
        "TÚQUERRES": ["TUQUERRES", "TÚQUERRES"]
    },
    "NORTE SANTANDER": {
        "ÁBREGO": ["ABREGO", "ÁBREGO"],
        "CÁCHIRA": ["CACHIRA", "CÁCHIRA"],
        "CÁCOTA": ["CACOTA", "CÁCOTA"],
        "CHINÁCOTA": ["CHINACOTA", "CHINÁCOTA"],
        "CHITAGÁ": ["CHITAGA", "CHITAGÁ"],
        "CONVENCIÓN": ["CONVENCION", "CONVENCIÓN"],
        "HACARÍ": ["HACARI", "HACARÍ"],
        "HERRÁN": ["HERRAN", "HERRÁN"],
        "TIBÚ": ["TIBU", "TIBÚ"]
    },
    "PUTUMAYO": {
        "COLÓN": ["COLON", "COLÓN"],
        "PUERTO ASÍS": ["PUERTO ASIS", "PUERTO ASÍS"],
        "PUERTO GUZMÁN": ["PUERTO GUZMAN", "PUERTO GUZMÁN"],
        "PUERTO LEGUÍZAMO": ["PUERTO\x92 LEGUIZAMO", "PUERTO LEGUÍZAMO"] # Se incluye la variante con el carácter especial
    },
    "QUINDIO": {
        "CALARCÁ": ["CALARCÁ"],
        "CÓRDOBA": ["CORDOBA", "CÓRDOBA"],
        "GÉNOVA": ["GENOVA", "GÉNOVA"]
    },
    "RISARALDA": {
        "APÍA": ["APIA", "APÍA"],
        "BELÉN DE UMBRÍA": ["BELEN DE UMBRIA", "BELÉN DE UMBRÍA"],
        "DOSQUEBRADAS": ["DOS QUEBRADAS", "DOSQUEBRADAS"],
        "GUÁTICA": ["GUATICA", "GUÁTICA"],
        "MISTRATÓ": ["MISTRATO", "MISTRATÓ"],
        "QUINCHÍA": ["QUINCHIA", "QUINCHÍA"]
    },
    "SAN ANDRES": {
        "SAN ANDRÉS": ["SAN ANDRES", "SAN ANDRÉS"]
    },
    "SANTANDER": {
        "BOLÍVAR": ["BOLIVAR", "BOLÍVAR"],
        "CARCASÍ": ["CARCASI", "CARCASÍ"],
        "CEPITÁ": ["CEPITA", "CEPITÁ"],
        "CHARALÁ": ["CHARALA", "CHARALÁ"],
        "CHIPATÁ": ["CHIPATA", "CHIPATÁ"],
        "CONTRATACIÓN": ["CONTRATACION", "CONTRATACIÓN"],
        "CURITÍ": ["CURITI", "CURITÍ"],
        "EL PEÑÓN": ["EL PEÑON", "EL PEÑÓN"],
        "EL PLAYÓN": ["EL PLAYON", "EL PLAYÓN"],
        "FLORIÁN": ["FLORIAN", "FLORIÁN"],
        "GALÁN": ["GALAN", "GALÁN"],
        "GA\x9cEPSA": ["GA\x9cEPSA", "GUEPSA"], # Conserva el carácter especial
        "GIRÓN": ["GIRON", "GIRÓN"],
        "GUAPOTÁ": ["GUAPOTA", "GUAPOTÁ"],
        "GUAVATÁ": ["GUAVATA", "GUAVATÁ"],
        "GÁMBITA": ["GAMBITA", "GÁMBITA"],
        "JESÚS MARÍA": ["JESUS MARIA", "JESÚS MARÍA"],
        "JORDÁN": ["JORDAN", "JORDÁN"],
        "LANDÁZURI": ["LANDAZURI", "LANDÁZURI"],
        "MÁLAGA": ["MALAGA", "MÁLAGA"],
        "PÁRAMO": ["PARAMO", "PÁRAMO"],
        "SAN ANDRÉS": ["SAN ANDRES", "SAN ANDRÉS"],
        "SAN JOAQUÍN": ["SAN JOAQUIN", "SAN JOAQUÍN"],
        "SAN JOSÉ DE MIRANDA": ["SAN JOSE DE MIRANDA", "SAN JOSÉ DE MIRANDA"],
        "SAN VICENTE DE CHUCURÍ": ["SAN VICENTE DE CHUCURI", "SAN VICENTE DE CHUCURÍ"],
        "SANTA BÁRBARA": ["SANTA BÁRBARA"],
        "SANTA HELENA DEL OPÓN": ["SANTA HELENA DEL OPON", "SANTA HELENA DEL OPÓN"],
        "SURATÁ": ["SURATA", "SURATÁ"],
        "VALLE DE SAN JOSÉ": ["VALLE DE SAN JOSE", "VALLE DE SAN JOSÉ"],
        "VÉLEZ": ["VELEZ", "VÉLEZ"]
    },
    "SUCRE": {
        "CHALÁN": ["CHALAN", "CHALÁN"],
        "COLOSÓ": ["COLOSO (RICAURTE)", "COLOSÓ"],
        "SAMPUÉS": ["SAMPUES", "SAMPUÉS"],
        "SINCÉ": ["SINCE", "SINCÉ"],
        "TOLÚ": ["TOLU", "TOLÚ"],
        "TOLÚ VIEJO": ["TOLUVIEJO", "TOLÚ VIEJO"]
    },
    "TOLIMA": {
        "ANZOÁTEGUI": ["ANZOATEGUI", "ANZOÁTEGUI"],
        "ARMERO": ["ARMERO", "ARMERO (GUAYABAL)"],
        "CARMEN DE APICALÁ": ["CARMEN DE APICALA", "CARMEN DE APICALÁ"],
        "IBAGUÉ": ["IBAGUE", "IBAGUÉ"],
        "LÉRIDA": ["LERIDA", "LÉRIDA"],
        "LÍBANO": ["LIBANO", "LÍBANO"],
        "PURIFICACIÓN": ["PURIFICACION", "PURIFICACIÓN"],
        "SUÁREZ": ["SUAREZ", "SUÁREZ"]
    },
    "VALLE DEL CAUCA": {
        "ALCALÁ": ["ALCALA", "ALCALÁ"],
        "ANDALUCÍA": ["ANDALUCIA", "ANDALUCÍA"],
        "BOLÍVAR": ["BOLIVAR", "BOLÍVAR"],
        "CALIMA": ["CALIMA", "CALIMA (DARIEN)"],
        "EL ÁGUILA": ["EL AGUILA", "EL ÁGUILA"],
        "GUACARÍ": ["GUACARI", "GUACARÍ"],
        "JAMUNDÍ": ["JAMUNDI", "JAMUNDÍ"],
        "RIOFRÍO": ["RIOFRIO", "RIOFRÍO"]
    },
    "VAUPES": {
        "MITÚ": ["MITU", "MITÚ"],
        "YAVARATÉ": ["YAVARATE", "YAVARATÉ"]
    },
    "VICHADA": {
        "SANTA ROSALÍA": ["SANTA ROSALIA", "SANTA ROSALÍA"]
    }
}
)

In [15]:
# Diccionario de municipios por departamento y sus variantes (proporcionado por el usuario)
# Asegurarse de que este diccionario 'municipios_dict' está definido en el entorno

# Paso 1: Crear un único diccionario de mapeo a partir de municipios_dict
# La clave será la variante en mayúsculas y sin espacios extras, el valor será el nombre estandarizado.
mapeo_estandarizacion_directa = {}
if 'municipios_dict' in locals() or 'municipios_dict' in globals():
    for departamento, municipios_en_depto in municipios_dict.items():
        if isinstance(municipios_en_depto, dict): # Asegurarse de que es un diccionario anidado
            for nombre_estandar, variantes in municipios_en_depto.items():
                if isinstance(variantes, list): # Asegurarse de que variantes es una lista
                    for variante in variantes:
                        if isinstance(variante, str):
                            # Usar una clave limpia (mayúsculas, sin espacios extras) para el mapeo
                            clave_mapeo = variante.upper().strip()
                            # Asegurarse de que el nombre estandarizado (valor) también esté limpio si es necesario
                            mapeo_estandarizacion_directa[clave_mapeo] = nombre_estandar # Usamos el nombre estandarizado tal cual del diccionario

else:
    print("[X] Error: El diccionario 'municipios_dict' no fue encontrado. Por favor, define la variable 'municipios_dict' en una celda anterior.")


# Paso 2: Aplicar el mapeo directamente a la columna 'cole_mcpio_ubicacion'
print("Estandarizando nombres de municipios en 'cole_mcpio_ubicacion' usando mapeo eficiente...")

# Crear una versión limpia (mayúsculas, sin espacios extras) de la columna para usarla en el mapeo
# Esto asegura que la coincidencia con las claves del mapeo funcione correctamente
columna_limpia = df_combinado['cole_mcpio_ubicacion'].astype(str).str.upper().str.strip()

# Usar .replace() en la columna limpia para obtener los valores mapeados
valores_mapeados = columna_limpia.replace(mapeo_estandarizacion_directa)

# Reemplazar los valores originales en 'cole_mcpio_ubicacion' con los valores mapeados
# Los valores que no encontraron coincidencia en el mapeo quedarán como estaban originalmente en la columna limpia
# Esto significa que si un valor original no estaba en el mapeo, su versión upper().strip() quedará en la columna
# Para mantener el formato original de los no mapeados, necesitamos un enfoque diferente.

# Enfoque alternativo para mantener el formato original de los no mapeados:
# Usar .map() en la columna limpia. Los valores que no encuentren coincidencia serán NaN.
# Luego, usar .fillna() para rellenar los NaN con los valores originales de la columna.
columna_mapeada_o_original = columna_limpia.map(mapeo_estandarizacion_directa).fillna(df_combinado['cole_mcpio_ubicacion'])

# Asignar el resultado a la columna original
df_combinado['cole_mcpio_ubicacion'] = columna_mapeada_o_original


print("Estandarización completa en la columna 'cole_mcpio_ubicacion'.")

# Mostrar las primeras filas para verificar los cambios en la columna original
display(df_combinado[['cole_mcpio_ubicacion', 'cole_depto_ubicacion']].head())

# Opcional: Contar cuántos valores cambiaron
# Esto es más complejo de contar directamente con .replace() o .map() + fillna
# Pero podemos contar cuántos valores únicos hay ahora
print(f"\nNúmero de municipios únicos en 'cole_mcpio_ubicacion' después de la estandarización: {df_combinado['cole_mcpio_ubicacion'].nunique(dropna=False)}")

# Opcional: Mostrar algunos ejemplos de municipios que *no* estaban en el diccionario de mapeo
# Para hacer esto, necesitamos identificar qué valores originales no se mapearon.
# Creamos una máscara: True si el valor original limpio NO está en las claves del mapeo
mask_no_mapeado = ~columna_limpia.isin(mapeo_estandarizacion_directa.keys())

print(f"\nNúmero de municipios en 'cole_mcpio_ubicacion' que NO estaban en el diccionario de mapeo: {mask_no_mapeado.sum()}")

if mask_no_mapeado.sum() > 0:
    print("\nEjemplos de municipios que no estaban en el diccionario de mapeo:")
    # Mostramos los valores originales de las filas que no se mapearon
    display(df_combinado.loc[mask_no_mapeado, 'cole_mcpio_ubicacion'].value_counts().head())

Estandarizando nombres de municipios en 'cole_mcpio_ubicacion' usando mapeo eficiente...
Estandarización completa en la columna 'cole_mcpio_ubicacion'.


,cole_mcpio_ubicacion,cole_depto_ubicacion
0,BUENAVENTURA,VALLE DEL CAUCA
1,MEDIO SAN JUAN,CHOCO
2,VILLAVICENCIO,META
3,CALI,VALLE DEL CAUCA
4,BOGOTÁ D.C.,BOGOTÁ D.C.



Número de municipios únicos en 'cole_mcpio_ubicacion' después de la estandarización: 1108

Número de municipios en 'cole_mcpio_ubicacion' que NO estaban en el diccionario de mapeo: 3966448

Ejemplos de municipios que no estaban en el diccionario de mapeo:


,count
cole_mcpio_ubicacion,
CALI,245139
BARRANQUILLA,183984
CARTAGENA DE INDIAS,113248
MEDELLIN,91259
SOACHA,85841


In [16]:
# Contar el número de valores únicos en la columna 'cole_mcpio_ubicacion'
num_municipios_cole_unicos = df_combinado['cole_mcpio_ubicacion'].nunique(dropna=False)

print(f"Número total de municipios únicos en la columna 'cole_mcpio_ubicacion': {num_municipios_cole_unicos}")

Número total de municipios únicos en la columna 'cole_mcpio_ubicacion': 1108


### Listar todos los Municipios únicos por Departamento (Columna `cole_mcpio_ubicacion`)

Este bloque de código agrupa el DataFrame por departamento (`cole_depto_ubicacion`) y lista todos los nombres de municipios únicos que aparecen en la columna `cole_mcpio_ubicacion` para cada departamento. Esto proporciona una vista completa de los municipios reportados por colegio, agrupados por su departamento.

In [17]:
# Agrupar por cole_depto_ubicacion y obtener los municipios únicos en cole_mcpio_ubicacion para cada grupo
municipios_por_departamento = df_combinado.groupby('cole_depto_ubicacion')['cole_mcpio_ubicacion'].unique()

# Mostrar la lista de municipios únicos por departamento
print("Municipios únicos por Departamento (columna 'cole_mcpio_ubicacion'):")
for depto, municipios in municipios_por_departamento.items():
    print(f"\n--- {depto} ---")
    # Convertir a string antes de ordenar para manejar NaNs u otros tipos
    print(sorted([str(item) for item in municipios]))

Municipios únicos por Departamento (columna 'cole_mcpio_ubicacion'):

--- AMAZONAS ---
['EL ENCANTO', 'LA CHORRERA', 'LA PEDRERA', 'LETICIA', 'MIRITI-PARANA', 'MIRITÍ - PARANÁ', 'PTO SANTANDER', 'PUERTO ARICA', 'PUERTO NARIÑO', 'TARAPACA', 'TARAPACÁ']

--- ANTIOQUIA ---
['ABEJORRAL', 'ABRIAQUI', 'ABRIAQUÍ', 'ALEJANDRÍA', 'AMAGA', 'AMAGÁ', 'AMALFI', 'ANDES', 'ANGELOPOLIS', 'ANGELÓPOLIS', 'ANGOSTURA', 'ANORI', 'ANORÍ', 'ANZA', 'ANZÁ', 'APARTADO', 'APARTADÓ', 'ARBOLETES', 'ARGELIA', 'ARMENIA', 'BARBOSA', 'BELLO', 'BELMIRA', 'BETANIA', 'BETULIA', 'BOLÍVAR', 'BRICEÑO', 'BURITICA', 'BURITICÁ', 'CACERES', 'CAICEDO', 'CALDAS', 'CAMPAMENTO', 'CARACOLI', 'CARACOLÍ', 'CARAMANTA', 'CAREPA', 'CARMEN DE VIBORAL', 'CAROLINA', 'CAUCASIA', 'CAÑASGORDAS', 'CHIGORODO', 'CHIGORODÓ', 'CISNEROS', 'CIUDAD BOLÍVAR', 'COCORNA', 'COCORNÁ', 'CONCEPCION', 'CONCEPCIÓN', 'CONCORDIA', 'COPACABANA', 'CÁCERES', 'DABEIBA', 'DON MATIAS', 'DONMATÍAS', 'EBEJICO', 'EBÉJICO', 'EL BAGRE', 'EL CARMEN DE VIBORAL', 'EL SANTUARI

In [18]:
# Mostrar las primeras filas del DataFrame df_combinado
display(df_combinado.head())

,estu_consecutivo,cole_nombre_establecimiento,cole_caracter,cole_naturaleza,cole_area_ubicacion,cole_calendario,cole_bilingue,cole_mcpio_ubicacion,cole_depto_ubicacion,periodo,estu_mcpio_presentacion,estu_depto_presentacion,punt_global,punt_lectura_critica,punt_matematicas,punt_ingles,punt_c_naturales,punt_sociales_ciudadanas,Año,Convocatoria
0,SB11201420348324,JUANCHACO,TÉCNICO/ACADÉMICO,OFICIAL,RURAL,A,NO,BUENAVENTURA,VALLE DEL CAUCA,20142,BUENAVENTURA,VALLE DEL CAUCA,274,61,47,41.0,53,63,2014,2
1,SB11201420159554,IE JOAQUIN URRUTIA,TÉCNICO/ACADÉMICO,OFICIAL,URBANO,A,NO,MEDIO SAN JUAN,CHOCO,20142,ISTMINA,CHOCO,222,40,45,48.0,43,48,2014,2
2,SB11201420457464,I. E. JUAN PABLO II,TÉCNICO/ACADÉMICO,OFICIAL,URBANO,A,NO,VILLAVICENCIO,META,20142,VILLAVICENCIO,META,358,72,74,62.0,71,73,2014,2
3,SB11201420593164,INSTITUTO DE EDUCACION TECNICA ICET CENTRO,TÉCNICO,NO OFICIAL,URBANO,A,NO,CALI,VALLE DEL CAUCA,20142,CALI,VALLE DEL CAUCA,184,35,36,46.0,29,44,2014,2
4,SB11201420554408,LIC SAN BERNARDO,ACADÉMICO,NO OFICIAL,URBANO,A,NO,BOGOTÁ D.C.,BOGOTÁ D.C.,20142,BOGOTÁ D.C.,BOGOTÁ D.C.,207,36,47,42.0,38,44,2014,2


### Paso Final: Guardar el DataFrame Limpio en un Archivo CSV

Este bloque de código guarda el DataFrame `df_combinado`, que ahora contiene los nombres de departamentos y municipios estandarizados, en un archivo CSV llamado `datos_icfes_2014_2024.csv`.

In [19]:
# Define el nombre del archivo CSV
nombre_archivo_salida = 'datos_icfes_2014_2024.csv'

# Guarda el DataFrame en un archivo CSV
# index=False evita escribir el índice del DataFrame como una columna en el CSV
df_combinado.to_csv(nombre_archivo_salida, index=False)

print(f"DataFrame guardado exitosamente como '{nombre_archivo_salida}'")

DataFrame guardado exitosamente como 'datos_icfes_2014_2024.csv'
